In [ ]:
%reset

In [46]:
import numpy as np
import pandas as pd
import random
import copy
from tqdm import tqdm
import os 


In [47]:
EXCESS_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv'
DEFICIT_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv'

In [48]:
excess = pd.read_csv(EXCESS_CSV_PATH)
deficit = pd.read_csv(DEFICIT_CSV_PATH)

excess['timestamp'] = pd.to_datetime(excess['timestamp'])
deficit['timestamp'] = pd.to_datetime(deficit['timestamp'])
excess['year_month'] = excess['timestamp'].dt.to_period('M')
deficit['year_month'] = deficit['timestamp'].dt.to_period('M')

excess_monthly = {}
deficit_monthly = {}

for i, month in enumerate(deficit['year_month'].unique()):

    excess_monthly[i] = excess[excess['year_month'] == month].drop('year_month', axis=1)
    deficit_monthly[i] = deficit[deficit['year_month'] == month].drop('year_month', axis=1)

In [49]:
month = 6

deficit_df = deficit_monthly[month]
excess_df = excess_monthly[month]

excess_df = excess_df.apply(pd.to_numeric, errors='coerce').fillna(0)
deficit_df = deficit_df.apply(pd.to_numeric, errors='coerce').fillna(0)

producers = list(excess_df.columns[1:])
consumers = list(deficit_df.columns[1:])

def generate_weights(producers, consumers):
    """
    select random consumer for each producer and then se
    """
    weights = {}
    for producer in producers:
        picked_cons = np.random.choice([c for c in consumers if c != producer], size=5, replace=False)
        assigned_weights = np.random.dirichlet(np.ones(len(picked_cons)), size=1)[0]  # Ensures sum = 1
        weights[producer] = dict(zip(picked_cons, assigned_weights))
    return pd.DataFrame(weights).fillna(0)

def generate_preferences(producers, consumers, n=5):
    preferences = {}
    for consumer in consumers:
        preferred_producers = np.random.choice([p for p in producers if p != consumer], size=n, replace=False)
        preference_values = np.random.permutation([i for i in range(1, n+1)])
        preferences[consumer] = dict(zip(preferred_producers, preference_values))
    return pd.DataFrame(preferences).fillna(0)

weights_df = generate_weights(producers, consumers)
preference_df = generate_preferences(producers, consumers)


def step(available_energy, remaining_demand, weights_df, preferences_df, rounds=5):
    producers = available_energy.index
    consumers = remaining_demand.index

    weights = weights_df.loc[consumers, producers].T.values
    preferences = preferences_df.loc[producers, consumers].values

    for _ in range(rounds):
        allocation = available_energy.values[:, None] * weights

        for j in range(len(consumers)):
            pref = preferences[:, j]
            sorted_idx = np.argsort(-pref)
            for i in sorted_idx:
                if pref[i] <= 0 or remaining_demand.iloc[j] <= 0:
                    continue
                transfer = min(allocation[i, j], remaining_demand.iloc[j])
                remaining_demand.iloc[j] -= transfer
                available_energy.iloc[i] -= transfer
                allocation[i, j] -= transfer

    return remaining_demand, available_energy

def allocation(weights_df, preferences_df, producer_energy_data, subscriber_demand_data):
    
    num_timesteps = len(producer_energy_data)
    results = np.zeros((num_timesteps, 3))

    prod_ids = producer_energy_data.columns[1:]
    cons_ids = subscriber_demand_data.columns[1:]

    for t in range(num_timesteps):
        ae = producer_energy_data.iloc[t, 1:].to_numpy(copy=True)
        rd = subscriber_demand_data.iloc[t, 1:].to_numpy(copy=True)

        ae[ae < 1e-9] = 0
        rd[rd < 1e-9] = 0

        ae_series = pd.Series(ae, index=prod_ids)
        rd_series = pd.Series(rd, index=cons_ids)

        rd_t, ae_t = step(ae_series, rd_series, weights_df, preferences_df)
        results[t] = [t, rd_t.sum(), ae_t.sum()]

    return pd.DataFrame(results, columns=['timestamp', 'unsatisfied_demand', 'available_energy'])


def fitness(weights_df, preferences_df, producer_energy_data, subscriber_demand_data):

    alloc_df = allocation(weights_df, preferences_df, producer_energy_data, subscriber_demand_data) 
    mask = (alloc_df['unsatisfied_demand'] > 0) & (alloc_df['available_energy'] > 0)
    return alloc_df.loc[mask, ['unsatisfied_demand', 'available_energy']].sum().sum()


Model Requirements:

1. **Decision Variables**:

- Weight matrix, according which producers distribute energy

1. **Constraints**:

- A consumer can only receive energy from producers they are connected to (preference > 0).
- A consumer can only receive energy up to their demand.
- A producer can only distribute up to their available supply.
- Respect the fixed limit of 5 connections per consumer.

1. **Objective Function**:

- Minimize the total unredistributed energy across all timestamps:
- Energy left with producers due to no eligible or matching demand.
- Unmet demand from consumers.


### Output:

- Total unredistributed energy over the full month.
- A breakdown of which consumers didn’t receive their full demand and which producers had unused energy. All of this output to a file for later analysis.

### Details:

Producers have weight vectors for distributing energy. Subscribers have preference vectors for up to 5 producers these preferences are given by users.The function evaluates the energy distribution for each timestamp, calculates unredistributed energy, and tracks remaining demand for each subscriber.



In [80]:
pref_jip = np.zeros((7, 7, 3), dtype=int)
pref_jip[0, 0, 0] = 1
pref_jip[0, 4, 1] = 1
pref_jip[0, 2, 2] = 1
pref_jip[1, 4, 0] = 1
pref_jip[1, 1, 1] = 1
pref_jip[1, 0, 2] = 1
pref_jip[2, 3, 0] = 1
pref_jip[2, 0, 1] = 1
pref_jip[2, 2, 2] = 1
pref_jip[3, 3, 0] = 1
pref_jip[3, 2, 1] = 1
pref_jip[3, 1, 2] = 1
pref_jip[4, 3, 0] = 1
pref_jip[4, 2, 1] = 1
pref_jip[4, 4, 2] = 1
pref_jip[5, 4, 0] = 1
pref_jip[5, 1, 1] = 1
pref_jip[5, 2, 2] = 1
pref_jip[6, 2, 0] = 1
pref_jip[6, 1, 1] = 1
pref_jip[6, 0, 2] = 1

pref_jip

array([[[1, 0, 0],
        [0, 0, 0],
        [0, 0, 1],
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 1, 0],
        [0, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [50]:
import gurobipy as gp
from gurobipy import GRB, quicksum


def optimalizace(excess_df, deficit_df, preference_df, rounds=5):
    """
    This model optimizes the redistribution of energy from entities with surplus
    to those with deficits. The same producer weight vector is used over all K time steps.
    
    Parameters:
      excess_df: pd.DataFrame
          Rows are time steps, columns are producer IDs. Each cell contains the available surplus.
      deficit_df: pd.DataFrame
          Rows are time steps, columns are consumer IDs. Each cell contains the energy deficit.
      preference_df: pd.DataFrame
          Rows are consumers, columns are producers. A zero means no connection; positive values (up to 5)
          indicate the consumer's preference for the producer.
      rounds: int (default=5)
          Number of redistribution rounds per time step.
          
    Returns:
      optimal_weights: dict (producer -> {consumer: weight})
      total_unmet_demand: float (sum of remaining unmet demand over all time steps)
      total_unused_supply: float (sum of remaining surplus over all time steps)
    """
    
    # Identify time steps, producers and consumers from the input DataFrames.
    time_steps = list(excess_df.index)
    K = len(time_steps)
    producers = list(excess_df.columns)
    consumers = list(deficit_df.columns)
    n_producers = len(producers)
    n_consumers = len(consumers)
    
    # Get the numpy arrays of available surplus and deficits.
    # supply[t, i] is the surplus energy for producer i at time step t.
    supply = excess_df.values   # shape: (K, n_producers)
    # demand[t, j] is the deficit for consumer j at time step t.
    demand = deficit_df.values  # shape: (K, n_consumers)
    
    # Preference matrix: rows = consumers, columns = producers.
    # A zero means no connection.
    pref = preference_df.to_numpy()  # shape: (n_consumers, n_producers)
    
    # Create the MILP model
    m = gp.Model("EnergyRedistribution")
    m.setParam('MIPGap', 0.03)
    m.setParam('Heuristics', 0.25)
    
    # ============================================================
    # 1. Producer Weight Variables (with cardinality constraints)
    # ============================================================
    # w[i,j] : fraction of energy produced by producer i offered to consumer j.
    # x[i,j] : binary variable linking the weight. If x[i,j]==1, then producer i assigns a nonzero weight to consumer j.
    w = {}
    x = {}
    for i in range(n_producers):
        for j in range(n_consumers):
            w[i, j] = m.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name=f"w_{i}_{j}")
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
    m.update()
    
    # Each producer's weights must sum to 1 and at most 5 weights are nonzero.
    for i in range(n_producers):
        m.addConstr(quicksum(w[i, j] for j in range(n_consumers)) == 1, name=f"weight_sum_{i}")
        m.addConstr(quicksum(x[i, j] for j in range(n_consumers)) <= 5, name=f"max_nonzero_{i}")
        for j in range(n_consumers):
            m.addConstr(w[i, j] <= x[i, j], name=f"link_{i}_{j}")
    
    # ====================================================================
    # 2. Multi-Round Flow and Residual Supply/Demand Variables per time step
    # ====================================================================
    # For each time step t (0 ... K-1) and each round r (0 ... rounds),
    # define:
    #   S[t,i,r]   : remaining supply for producer i after round r (r=0 means initial supply).
    #   D[t,j,r]   : remaining demand for consumer j after round r (r=0 means initial demand).
    #   f[t,i,j,r] : energy flow from producer i to consumer j in round r.
    #
    # Initial conditions:
    #   S[t,i,0] = supply[t, i]
    #   D[t,j,0] = demand[t, j]
    #
    # For each round r>=1:
    #   f[t,i,j,r] <= w[i,j] * S[t,i,r-1]       (producers offer energy based on current available surplus)
    #   f[t,i,j,r] <= D[t,j,r-1]                  (consumer j cannot accept more than remaining deficit)
    #   Additionally, if pref[j,i] == 0 then f[t,i,j,r] must equal 0.
    #
    # Then update:
    #   S[t,i,r] = S[t,i,r-1] - sum_j f[t,i,j,r]
    #   D[t,j,r] = D[t,j,r-1] - sum_i f[t,i,j,r]
    
    S = {}
    D = {}
    f = {}
    
    # Set initial conditions for all time steps.
    for t in range(K):
        for i in range(n_producers):
            S[t, i, 0] = supply[t, i]
        for j in range(n_consumers):
            D[t, j, 0] = demand[t, j]
    
    # Define variables for rounds (round index 1 to rounds)
    for t in range(K):
        for r in range(1, rounds + 1):
            for i in range(n_producers ):
                S[t, i, r] = m.addVar(lb=0.0, name=f"S_{t}_{i}_{r}")
            for j in range(n_consumers ):
                D[t, j, r] = m.addVar(lb=0.0, name=f"D_{t}_{j}_{r}")
            for i in range(n_producers):
                for j in range(n_consumers):
                    f[t, i, j, r] = m.addVar(lb=0.0, name=f"f_{t}_{i}_{j}_{r}")
    m.update()
    
    # ======================================================
    # 3. Flow and Update Constraints over time steps & rounds
    # ======================================================


    for t in range(K):
        for r in range(1, rounds + 1):
            for i in range(n_producers):
                for j in range(n_consumers):
                    # If no connection (preference zero) then no flow is allowed.
                    if pref[j, i] == 0.0:
                        continue
                        #m.addConstr(f[t, i, j, r] == 0, name=f"pref_zero_{t}_{i}_{j}_{r}")
                    else:
                        m.addConstr(f[t, i, j, r] <= w[i, j] * S[t, i, r-1], name=f"flow_supply_{t}_{i}_{j}_{r}")
                        m.addConstr(f[t, i, j, r] <= D[t, j, r-1], name=f"flow_demand_{t}_{i}_{j}_{r}")
            # Update remaining supply for each producer.
            for i in range(n_producers):
                m.addConstr(S[t, i, r] == S[t, i, r-1] - quicksum(f[t, i, j, r] for j in range(n_consumers)),
                            name=f"update_supply_{t}_{i}_{r}")
            # Update remaining demand for each consumer.
            for j in range(n_consumers):
                m.addConstr(D[t, j, r] == D[t, j, r-1] - quicksum(f[t, i, j, r] for i in range(n_producers)),
                            name=f"update_demand_{t}_{j}_{r}")
    
    # =====================================
    # 4. Objective: Minimize left-over energy
    # =====================================
    # We minimize the sum (over all time steps) of:
    #   - The remaining supply after the final round (unused surplus)
    #   - The remaining demand after the final round (unmet demand)
    obj_unused_supply = quicksum(S[t, i, rounds] for t in range(K) for i in range(n_producers))
    obj_unmet_demand  = quicksum(D[t, j, rounds] for t in range(K) for j in range(n_consumers))
    m.setObjective(obj_unused_supply + obj_unmet_demand, GRB.MINIMIZE)
    
    m.params.NonConvex = 2  # Allows for certain nonconvexities that may arise.
    
    # =====================================
    # 5. Solve the model and output results
    # =====================================
    m.optimize()
    
    # Extract optimal weights for each producer.
    optimal_weights = {}
    for i in range(n_producers):
        prod_id = producers[i]
        optimal_weights[prod_id] = {}
        for j in range(n_consumers):
            optimal_weights[prod_id][consumers[j]] = w[i, j].X

    total_unused_supply = sum(S[t, i, rounds].X for t in range(K) for i in range(n_producers))
    total_unmet_demand  = sum(D[t, j, rounds].X for t in range(K) for j in range(n_consumers))
    
    print("Results after redistribution:")
    print("Total Unmet Demand:", total_unmet_demand)
    print("Total Unused Supply:", total_unused_supply)
    print("Optimal Producer Weights:")
    for prod, weights in optimal_weights.items():
        print(f"Producer {prod}: {weights}")
    
    return optimal_weights, total_unmet_demand, total_unused_supply

In [67]:
optimal_weights, total_unmet_demand, total_unused_supply = optimalizace(excess_df, deficit_df, preference_df, rounds=5)

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: AMD Ryzen 3 4300U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
NonConvex  2

Optimize a model with 1205694 rows, 1134952 columns and 3588830 nonzeros
Model fingerprint: 0x669b9725
Model has 283640 quadratic constraints
Variable types: 1134756 continuous, 196 integer (196 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-07, 5e+00]
Presolve removed 1205470 rows and 794192 columns
Presolve time: 2.96s
Presolved: 567504 rows, 340761 columns, 851704 nonzeros
Presolved model has 283640 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 340565 continuous, 196 integer (19

In [56]:
month = 4

deficit_df = deficit_monthly[month]
excess_df = excess_monthly[month]

excess_df = excess_df.apply(pd.to_numeric, errors='coerce').fillna(0)
deficit_df = deficit_df.apply(pd.to_numeric, errors='coerce').fillna(0)

excess_df = excess_df.drop(columns='timestamp').reset_index(drop=True)
deficit_df = deficit_df.drop(columns='timestamp').reset_index(drop=True)
excess_df['sum'] = excess_df.sum(axis=1)
deficit_df['sum'] = deficit_df.sum(axis=1)

deficit_df = deficit_df.iloc[excess_df[excess_df['sum'] >= 0.0].index].reset_index(drop=True)
excess_df = excess_df[excess_df['sum'] >= 0.0].reset_index(drop=True)

deficit_df = deficit_df.drop(columns='sum')
excess_df = excess_df.drop(columns='sum')

In [57]:
preference_df = preference_df.loc[excess_df.columns, excess_df.columns]

In [58]:
preference_df

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
zs_preislerova,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0
zs_komenskeho,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,2.0,2.0,2.0,0.0,4.0,4.0
ms_preislerova,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
ms_pod_homolkou,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
ms_vrchlickeho,0.0,5.0,5.0,5.0,0.0,2.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
ms_drasarova,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,5.0,0.0,4.0,2.0,5.0
ms_na_machovne,3.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
zimni_stad,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
parkovaci_dum,2.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,2.0,5.0,0.0
radnice,0.0,0.0,0.0,4.0,4.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0


In [67]:
def prepare_data(excess_monthly, deficit_monthly, month):
    deficit_df = deficit_monthly[month]
    excess_df = excess_monthly[month]

    excess_df = excess_df.apply(pd.to_numeric, errors='coerce').fillna(0)
    deficit_df = deficit_df.apply(pd.to_numeric, errors='coerce').fillna(0)

    excess_df = excess_df.drop(columns='timestamp').reset_index(drop=True)
    deficit_df = deficit_df.drop(columns='timestamp').reset_index(drop=True)
    excess_df['sum'] = excess_df.sum(axis=1)
    deficit_df['sum'] = deficit_df.sum(axis=1)

    deficit_df = deficit_df.iloc[excess_df[excess_df['sum'] > 0.0].index].reset_index(drop=True)
    excess_df = excess_df[excess_df['sum'] > 0.0].reset_index(drop=True)

    deficit_df = deficit_df.drop(columns='sum')
    excess_df = excess_df.drop(columns='sum')

    return excess_df, deficit_df

In [68]:
excess_monthly[4]

,timestamp,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
11516,2023-05-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
11517,2023-05-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
11518,2023-05-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
11519,2023-05-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
11520,2023-05-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14487,2023-05-31 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
14488,2023-05-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
14489,2023-05-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
14490,2023-05-31 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0


In [71]:
excess_df, deficit_df = prepare_data(excess_monthly, deficit_monthly, 4)
excess_df = excess_df.iloc[4:10]
deficit_df = deficit_df.iloc[4:10]

In [72]:
optimal_weights, total_unmet_demand, total_unused_supply = optimalizace(excess_df, deficit_df, preference_df, rounds=5)

Set parameter MIPGap to value 0.03
Set parameter Heuristics to value 0.25
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: AMD Ryzen 3 4300U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
MIPGap  0.03
Heuristics  0.25
NonConvex  2

Optimize a model with 3584 rows, 7112 columns and 18541 nonzeros
Model fingerprint: 0x4a1f6481
Model has 1680 quadratic constraints
Variable types: 6916 continuous, 196 integer (196 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-05, 5e+00]
Presolve removed 2951 rows and 5290 columns
Presolve time: 0.03s
Presolved: 1569 rows, 1823 columns, 5793 nonzeros
Presolved model has 468 bilinear constraint(s)


In [75]:
weights = pd.DataFrame.from_dict(optimal_weights)

fitness(weights , preference_df, excess_df, deficit_df)

np.float64(0.7536577684080903)

In [ ]:
res = {}
import time



for month in range(12):

    excess_df, deficit_df = prepare_data(excess_monthly, deficit_monthly, month)

    start = time.time()
    optimal_weights, total_unmet_demand, total_unused_supply = optimalizace(excess_df, deficit_df, preference_df, rounds=5)
    end = time.time()
    res[month] = [optimal_weights, total_unmet_demand, total_unused_supply, end - start]
    print(f'done for {month}')

Set parameter MIPGap to value 0.03
Set parameter Heuristics to value 0.25


Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: AMD Ryzen 3 4300U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
MIPGap  0.03
Heuristics  0.25
NonConvex  2

Optimize a model with 1666784 rows, 3333512 columns and 8693336 nonzeros
Model fingerprint: 0x8462fd3a
Model has 833280 quadratic constraints
Variable types: 3333316 continuous, 196 integer (196 binary)
Coefficient statistics:
  Matrix range     [5e-08, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-08, 5e+00]
Presolve removed 1533081 rows and 3095156 columns
Presolve time: 2.48s
Presolved: 253343 rows, 238357 columns, 854627 nonzeros
Presolved model has 59820 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2381

In [40]:
res

{0: [{'zs_preislerova': {'zs_preislerova': 0.0,
    'zs_komenskeho': 0.0,
    'ms_preislerova': 0.0,
    'ms_pod_homolkou': 0.0,
    'ms_vrchlickeho': 0.0,
    'ms_drasarova': 0.0,
    'ms_na_machovne': 0.0,
    'zimni_stad': 0.0,
    'parkovaci_dum': 0.0,
    'radnice': 0.0,
    'ms_tovarni': 0.0,
    'dum_pro_duchodce': 0.0,
    'plavecky_areal': 0.0,
    'pristavba_preislerova': 1.0},
   'zs_komenskeho': {'zs_preislerova': 0.0,
    'zs_komenskeho': 0.0,
    'ms_preislerova': 0.0,
    'ms_pod_homolkou': 0.0,
    'ms_vrchlickeho': 0.0,
    'ms_drasarova': 0.0,
    'ms_na_machovne': 0.0,
    'zimni_stad': 0.0,
    'parkovaci_dum': 0.0,
    'radnice': 0.0,
    'ms_tovarni': 0.0,
    'dum_pro_duchodce': 0.0,
    'plavecky_areal': 0.0,
    'pristavba_preislerova': 1.0},
   'ms_preislerova': {'zs_preislerova': 0.0,
    'zs_komenskeho': 0.0,
    'ms_preislerova': 0.0,
    'ms_pod_homolkou': 0.0,
    'ms_vrchlickeho': 0.0,
    'ms_drasarova': 0.0,
    'ms_na_machovne': 0.0,
    'zimni_stad':

In [41]:

for month in range(12):

    excess_df, deficit_df = prepare_data(excess_monthly, deficit_monthly, month)
    weights = pd.DataFrame.from_dict(res[month][0])

    res[month].append(fitness(weights , preference_df, excess_df, deficit_df))

In [42]:
res[5][1:]

[63.16481667142126,
 117.26290272743876,
 130.53782629966736,
 np.float64(138.67247786221364)]

In [44]:
res[6][1:]

[99.99373364574369,
 120.89071409354214,
 126.22302031517029,
 np.float64(264.9997930433079)]